# Bonsai Interview Test

Scenario:
You joined as the new memeber of a small start-up team. Together we are building a new app to sell cool 3rd party products! So far, the sales team worked tirelessly and managed to acquire over 100 merchants who each have different brands and products offerings. The developers have made a ton of progress on the mobile app and there's a bunch of user activity on the app. The next step is to optimize user conversion rates by offering new recommendations based on the analytics data.

Goal:
Your task is to recommend a book that a user is most likely to buy next using the data provided. You can find the dataset here: https://www.dropbox.com/sh/uj3nsf66mtwm36q/AADLUNVShEZ0VI3DsLad6S4Ta?dl=0

In [1]:
#Making necesarry imports
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets
from IPython.display import display, clear_output
from contextlib import contextmanager
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os, sys
import re
import seaborn as sns
from math import sqrt

# Starting Exploratory Data Analysis

In [2]:
#Setting the current working directory
os.chdir('D:/userdata/girims/Downloads/ShopBonsai')

In [3]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [4]:
# Reading users file
users = pd.read_csv('Users.csv', encoding='latin-1')

#Reading books file
books = pd.read_csv('Books.csv', encoding='latin-1')

#Reading events file
events = pd.read_csv('UserEvents.csv', encoding='latin-1')

In [5]:
#checking shapes of the datasets before data pre-processing
print(users.shape)
print(books.shape)
print(events.shape)

(1000, 5)
(3000, 8)
(141081, 4)


In [6]:
#making this setting to display full text in columns
pd.set_option('display.max_colwidth', -1)

In [7]:
#Exploring users dataset
users.head()

,Unnamed: 0,Unnamed: 0.1,user,location,age
0,57300,43156,43157.0,"windsor, ontario, canada",22.0
1,40638,99031,99032.0,"white plains, new york, usa",25.0
2,54265,42882,42883.0,"the woodlands, texas, usa",51.0
3,83797,183799,183800.0,"quebec, quebec, canada",29.0
4,96743,188795,188796.0,"toronto, ontario, canada",NaN


In [8]:
#Looks like there are unwanted columns in users dataset, let's drop them.
users =  users.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)
users.head()

,user,location,age
0,43157.0,"windsor, ontario, canada",22.0
1,99032.0,"white plains, new york, usa",25.0
2,42883.0,"the woodlands, texas, usa",51.0
3,183800.0,"quebec, quebec, canada",29.0
4,188796.0,"toronto, ontario, canada",NaN


In [9]:
#Exploring books dataset
books.head()

,Unnamed: 0,Unnamed: 0.1,bookISBN,bookName,author,yearOfPublication,publisher,urlId
0,62044,26575,1551669382,Enchanted Afternoon,Susan Wiggs,2002,Mira,120656.0
1,85751,1995,0743202961,"Every Breath You Take : A True Story of Obsession, Revenge, and Murder",Ann Rule,2001,Free Press,116803.0
2,122688,7677,067151699X,The HIDDEN LIFE OF DOGS,Elizabeth Marshall Thomas,1995,Pocket,1805.0
3,87360,36120,0804108528,Inadmissible Evidence,Philip Friedman,1993,Ivy Books,139375.0
4,73712,30238,0061094145,"The Fifth Dominion (Imajica, Book 1)",Clive Barker,1995,HarperTorch,141585.0


In [10]:
#Looks like there are unwanted columns in books dataset, let's drop them.
books =  books.drop(['Unnamed: 0','Unnamed: 0.1','urlId'], axis=1)
books.head()

,bookISBN,bookName,author,yearOfPublication,publisher
0,1551669382,Enchanted Afternoon,Susan Wiggs,2002,Mira
1,0743202961,"Every Breath You Take : A True Story of Obsession, Revenge, and Murder",Ann Rule,2001,Free Press
2,067151699X,The HIDDEN LIFE OF DOGS,Elizabeth Marshall Thomas,1995,Pocket
3,0804108528,Inadmissible Evidence,Philip Friedman,1993,Ivy Books
4,0061094145,"The Fifth Dominion (Imajica, Book 1)",Clive Barker,1995,HarperTorch


In [11]:
#Exploring events dataset
events.head()

,Unnamed: 0,user,bookId,impression
0,16,276747,0060517794,checkout
1,19,276747,0671537458,checkout
2,20,276747,0679776818,add to cart
3,133,276822,0060096195,checkout
4,137,276822,0375821813,checkout


In [12]:
#Looks like there are unwanted columns in events dataset, let's drop them.
events =  events.drop(['Unnamed: 0'], axis=1)
events.head()

,user,bookId,impression
0,276747,0060517794,checkout
1,276747,0671537458,checkout
2,276747,0679776818,add to cart
3,276822,0060096195,checkout
4,276822,0375821813,checkout


In [13]:
#Let's make the column names uniform across the dataset for easier processing. For example bookId field in events dataset would be changed to bookISBN.
events.columns = ['user', 'bookISBN','impression']
events.head()

,user,bookISBN,impression
0,276747,0060517794,checkout
1,276747,0671537458,checkout
2,276747,0679776818,add to cart
3,276822,0060096195,checkout
4,276822,0375821813,checkout


In [14]:
#checking shapes of the datasets after dropping unwanted columns
print(users.shape)
print(books.shape)
print(events.shape)

(1000, 3)
(3000, 5)
(141081, 3)


In [15]:
#Our Users dataset contains 1000 unique entries for users and Books dataset contains 3000 unique entries for books.
#Let's see the unique entries users and books from UserEvents dataset.
n_users = events.user.unique().shape[0]
n_books = events.bookISBN.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of books = ' + str(n_books))

Number of users = 13030 | Number of books = 11234


In [16]:
users.dtypes

user        float64
location    object 
age         float64
dtype: object

In [17]:
books.dtypes

bookISBN             object
bookName             object
author               object
yearOfPublication    int64 
publisher            object
dtype: object

In [18]:
events.dtypes

user          int64 
bookISBN      object
impression    object
dtype: object

As mentioned earlier we could see only bookISBN, user and impression are the only variables we can rely on to build the recommender system. But here is the "user "datatype mismatch in users dataset and events dataset. So let's convert the "user" datatype from float64 to int64.

In [19]:
#Let's convert user id from float to integer
users.user = users.user.astype(np.int64)

In [20]:
users.dtypes

user        int64  
location    object 
age         float64
dtype: object

In [21]:
events.dtypes

user          int64 
bookISBN      object
impression    object
dtype: object

Now the mismatch of variable datatype is resolved(Current datatype : user -> int64, bookISBN -> object, impression -> object)

Let us check all the available three datasets for missing values using Pandas profiling method.

In [22]:
pandas_profiling.ProfileReport(users)

Number of variables,3
Number of observations,1000
Total Missing (%),10.9%
Total size in memory,23.5 KiB
Average record size in memory,24.1 B
Numeric,2
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


We could see lot of missing values for variables(location, age). We can delete them and proceed further in building the recommender system, but to make better decisions they might be helpful. By using Google analytics, if we are able to fetch the data of unregistered users as well, then we can build better recommender systems which might be based on customer location, age etc.,

Below are the few advantages of using Google Analytics:

1. See your most important analytics data first
2. Find out which online campaigns bring the most traffic and conversions
3. Determine where your best visitors are located
4. Learn what people are searching for on your site
5. Visualize what people click on the most
6. Uncover your top content
7. Identify your worst performing pages
8. Determine where people abandon the shopping cart

I am going to use dummy values, which can be replaced with actual ones from Google Analytics data

In [23]:
#Let's see how many young readers among the Users dataset :-)
print(sorted(users.age.unique()))

[12.0, 18.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 34.0, 39.0, 41.0, 43.0, 47.0, 51.0, nan, 0.0, 2.0, 3.0, 4.0, 8.0, 9.0, 13.0, 14.0, 15.0, 16.0, 17.0, 19.0, 20.0, 33.0, 35.0, 36.0, 37.0, 38.0, 40.0, 42.0, 44.0, 45.0, 46.0, 48.0, 49.0, 50.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 74.0, 76.0, 81.0, 103.0, 209.0]


Age column has some invalid entries like nan, and surprising age groups such as <5 and > 80.

In [24]:
#In my view values below 5 and above 80 do not make much sense for our book rating case...hence replacing these by NaNs
users.loc[(users.age > 80) | (users.age < 5), 'age'] = np.nan
print(sorted(users.age.unique()))

[8.0, 9.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 34.0, 37.0, 39.0, 41.0, 43.0, 51.0, nan, 33.0, 35.0, 36.0, 38.0, 40.0, 42.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 74.0, 76.0]


In [25]:
#user age, will use my age as default value
users.age = users.age.fillna(29)
#Let's set the data type as int
users.age = users.age.astype(np.int32)

In [26]:
#Let's see the age distribution once, can relate to reality?
users.age.hist(bins=[0,20,40,60,80,100])
plt.show()

In [27]:
#user location, will use Canada by default
users.location = users.location.fillna('Canada')

In [28]:
#Now Check for missing values in Users dataset
users.isnull().sum()

user        0
location    0
age         0
dtype: int64

In [29]:
users.head()

,user,location,age
0,43157,"windsor, ontario, canada",22
1,99032,"white plains, new york, usa",25
2,42883,"the woodlands, texas, usa",51
3,183800,"quebec, quebec, canada",29
4,188796,"toronto, ontario, canada",29


In [30]:
pandas_profiling.ProfileReport(books)

Number of variables,5
Number of observations,3000
Total Missing (%),0.7%
Total size in memory,117.3 KiB
Average record size in memory,40.0 B
Numeric,1
Categorical,3
Boolean,0
Date,0
Text (Unique),1
Rejected,0


In [31]:
#author name, will use Canada's famous author name "Margaret Atwood"
books.author = books.author.fillna('Margaret Atwood')

In [32]:
#book name, will use "The Robber Bride" as the default book name
books.bookName = books.bookName.fillna('The Robber Bride')

In [33]:
print(sorted(books.yearOfPublication.unique()))

[0, 1920, 1927, 1929, 1942, 1946, 1951, 1952, 1953, 1954, 1955, 1958, 1960, 1961, 1963, 1964, 1965, 1966, 1967, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005]


In [34]:
#Year of Publication, will use my year of birth(1989) as default value
books.loc[books.yearOfPublication == 0,'yearOfPublication'] = 1989

In [35]:
#Publisher, will use Goyal publishers as default value
books.publisher = books.publisher.fillna('Goyal Publishers')

In [36]:
#Now Check for missing values in Books dataset
books.isnull().sum()

bookISBN             0
bookName             0
author               0
yearOfPublication    0
publisher            0
dtype: int64

In [37]:
pandas_profiling.ProfileReport(events)

Number of variables,3
Number of observations,141081
Total Missing (%),0.0%
Total size in memory,3.2 MiB
Average record size in memory,24.0 B
Numeric,1
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [38]:
#checking first few rows...
events.head(5)

,user,bookISBN,impression
0,276747,0060517794,checkout
1,276747,0671537458,checkout
2,276747,0679776818,add to cart
3,276822,0060096195,checkout
4,276822,0375821813,checkout


In [39]:
#UserEvents dataset will have n_users*n_books entries if every user rated every item, this shows that the dataset is very sparse
n_users = users.shape[0]
n_books = books.shape[0]
print(n_users * n_books)

3000000


In [40]:
#UserEvents dataset should have books only which exist in our Books dataset, unless new books are added to Books dataset.
new_events = events[events.bookISBN.isin(books.bookISBN)]

In [41]:
print(events.shape)
print(new_events.shape)
#it can be seen that many rows having book ISBN not part of books dataset got dropped off

(141081, 3)
(37546, 3)


In [42]:
#UserEvents dataset should have impressions/interactions from users which exist in Users dataset, unless new users are added to Users dataset
events = events[events.user.isin(users.user)]

In [43]:
print(events.shape)
print(new_events.shape)
#no new users added, hence we will go with above dataset ratings_new (37546, 3)

(10908, 3)
(37546, 3)


In [44]:
print("number of users: " + str(n_users))
print("number of books: " + str(n_books))

number of users: 1000
number of books: 3000


In [45]:
#Sparsity of dataset in %
sparsity=1.0-len(new_events)/float(n_users*n_books)
print('The sparsity level of ShopBonsai dataset is ' +  str(sparsity*100) + ' %')

The sparsity level of ShopBonsai dataset is 98.74846666666667 %


In [46]:
new_events.head()

,user,bookISBN,impression
9,276847,3423071516,checkout
13,276847,3442446414,checkout
16,276847,3551551685,checkout
20,276859,0836218833,checkout
29,276925,002542730X,checkout


In [47]:
#Now Check for missing values in UserEvents(new) dataset
new_events.isnull().sum()

user          0
bookISBN      0
impression    0
dtype: int64

In [48]:
new_events.impression.unique()

array(['checkout', 'dislike', 'like', 'add to cart', 'view', 'interact'],
      dtype=object)

In [49]:
#Convert the 'impression' in new_events dataset from categorical values to numerical values.
new_events["impression"] = new_events["impression"].astype('category')
new_events.dtypes

user          int64   
bookISBN      object  
impression    category
dtype: object

In [50]:
codes = {'dislike':1, 'view':2, 'interact':3, 'like':4, 'add to cart':5, 'checkout':6}

In [51]:
new_events['impression_codes'] = new_events['impression'].map(codes)

In [52]:
sns.countplot(data=new_events, x='impression_codes')
plt.show()

In [53]:
display(new_events[:10])

,user,bookISBN,impression,impression_codes
9,276847,3423071516,checkout,6
13,276847,3442446414,checkout,6
16,276847,3551551685,checkout,6
20,276859,0836218833,checkout,6
29,276925,002542730X,checkout,6
38,276929,2266104535,dislike,1
40,276939,2253063339,checkout,6
53,276964,0886773741,like,4
54,276964,0886774802,add to cart,5
56,276964,0886775426,checkout,6


In [54]:
new_events.impression_codes.unique()

array([6, 1, 4, 5, 2, 3], dtype=int64)

In [55]:
#Will see what happens if we segregate "dislike" impressions
impressions_positive = new_events[new_events.impression_codes >= 2]
impressions_negative = new_events[new_events.impression_codes == 1]

In [56]:
#checking shapes
print(new_events.shape)
print(impressions_positive.shape)
print(impressions_negative.shape)

(37546, 4)
(36177, 4)
(1369, 4)


# Simple Popularity Based Recommendation System

In [57]:
#At this point, a simple popularity based recommendation system can be built based on count of user impression for different books
impressions_count = pd.DataFrame(impressions_positive.groupby(['bookISBN'])['impression_codes'].sum())
Most_popular_books = impressions_count.sort_values('impression_codes', ascending = False).head(10)
print("<------ Following books can be recommended for any user ------>")
Most_popular_books.merge(books, left_index = True, right_on = 'bookISBN')

#Given below are top 10 recommendations based on popularity, J.K.Rowling looks to be the most popular author..!!

<------ Following books can be recommended for any user ------>


,impression_codes,bookISBN,bookName,author,yearOfPublication,publisher
1522,1249,059035342X,Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),J. K. Rowling,1999,Arthur A. Levine Books
79,1242,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA
2008,996,0446672211,Where the Heart Is (Oprah's Book Club (Paperback)),Billie Letts,1998,Warner Books
1388,930,043935806X,Harry Potter and the Order of the Phoenix (Book 5),J. K. Rowling,2003,Scholastic
1323,829,0156027321,Life of Pi,Yann Martel,2003,Harvest Books
943,798,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion
175,763,044021145X,The Firm,John Grisham,1992,Bantam Dell Publishing Group
1444,738,0345370775,Jurassic Park,Michael Crichton,1999,Ballantine Books
855,669,0439139597,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic
1931,647,1400034779,The No. 1 Ladies' Detective Agency (Today Show Book Club #8),Alexander McCall Smith,2003,Anchor


In [58]:
#Similarly segregating users who have "dislike" impressions over books
users_positive_impressions = users[users.user.isin(impressions_positive.user)]
users_negative_impressions = users[users.user.isin(impressions_negative.user)]

In [59]:
#checking shapes, this is just for static analysis
print(users.shape)
print(users_positive_impressions.shape)
print(users_negative_impressions.shape)

(1000, 3)
(738, 3)
(82, 3)


# Collaborative Filtering Based Recommendation System

In [60]:
#To cope up with computing power I have and to reduce the dataset size, I am considering users who have rated atleast 50 books
#and books which have atleast 50 ratings
min_user_count = impressions_positive['user'].value_counts()
impressions_positive = impressions_positive[impressions_positive['user'].isin(min_user_count[min_user_count >= 50].index)]
min_impression_count = impressions_positive['impression_codes'].value_counts()
impressions_positive = impressions_positive[impressions_positive['impression_codes'].isin(min_impression_count[min_impression_count >= 50].index)]

In [61]:
impressions_positive.head()

,user,bookISBN,impression,impression_codes
3763,7346,0060507772,checkout,6
3767,7346,0061043508,add to cart,5
3775,7346,0140253602,add to cart,5
3776,7346,0140254544,add to cart,5
3779,7346,0140430725,checkout,6


In [62]:
#Generating impressions matrix from impressions_positive table
impressions_matrix = impressions_positive.pivot(index='user', columns='bookISBN', values='impression_codes')
user = impressions_matrix.index
bookISBN = impressions_matrix.columns
print(impressions_matrix.shape)
impressions_matrix.head()
#Notice that most of the values are NaN (undefined) implying absence of impressions.

(20, 1513)


bookISBN,0002005018,000648302X,0006550924,0007110928,0007154615,0020198817,002026478X,0020518501,002542730X,0060001453,...,3442453305,3442455707,3442720117,3462028189,3518100734,3548202020,3596215226,3596247500,8807813025,9068340344
user,,,,,,,,,,,,,,,,,,,,,
7346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11676,5.0,5.0,NaN,4.0,4.0,NaN,NaN,6.0,3.0,NaN,...,5.0,4.0,5.0,5.0,6.0,6.0,2.0,2.0,6.0,2.0
16795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23902,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
n_users = impressions_matrix.shape[0] #considering only those users who had an impression other than "dislike"
n_books = impressions_matrix.shape[1]
print(n_users,':', n_books)

20 : 1513


In [64]:
#since NaNs cannot be handled by training algorithms, replacing these by 0, which indicates absence of ratings
#setting data type
impressions_matrix.fillna(0, inplace = True)
impressions_matrix = impressions_matrix.astype(np.int32)

In [65]:
#checking first few rows
impressions_matrix.head(5)

bookISBN,0002005018,000648302X,0006550924,0007110928,0007154615,0020198817,002026478X,0020518501,002542730X,0060001453,...,3442453305,3442455707,3442720117,3462028189,3518100734,3548202020,3596215226,3596247500,8807813025,9068340344
user,,,,,,,,,,,,,,,,,,,,,
7346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11676,5,5,0,4,4,0,0,6,3,0,...,5,4,5,5,6,6,2,2,6,2
16795,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23902,0,0,5,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35859,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
#rechecking the sparsity
sparsity=1.0-len(impressions_positive)/float(users_positive_impressions.shape[0]*n_books)
print('The sparsity level of Book Crossing dataset is ' +  str(sparsity*100) + ' %')

The sparsity level of Book Crossing dataset is 99.8006437433839 %


# Training our recommendation system

In [67]:
#setting global variables
global metric,k
k=5
metric='cosine'

# User-based Recommendation System

In [68]:
#This function finds k similar users given the user_id and impressions matrix 
#These similarities are same as obtained via using pairwise_distances
def findksimilarusers(user_id, impressions, metric = metric, k=k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
    model_knn.fit(impressions)
    loc = impressions.index.get_loc(user_id)
    distances, indices = model_knn.kneighbors(impressions.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
            
    return similarities,indices

In [69]:
#This function predicts impression for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, impressions, metric = metric, k=k):
    prediction=0
    user_loc = impressions.index.get_loc(user_id)
    item_loc = impressions.columns.get_loc(item_id)
    similarities, indices=findksimilarusers(user_id,impressions,metric, k) #similar users based on cosine similarity
    mean_impression = impressions.iloc[user_loc,:].mean() #to adjust for zero based indexing
    sum_wt = np.sum(similarities)-1
    product=1
    wtd_sum = 0 
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] == user_loc:
            continue;
        else: 
            impressions_diff = impressions.iloc[indices.flatten()[i],item_loc]-np.mean(impressions.iloc[indices.flatten()[i],:])
            product = impressions_diff * (similarities[i])
            wtd_sum = wtd_sum + product
    
    #in case of very sparse datasets, using correlation metric for collaborative based approach may give negative ratings
    #which are handled here as below
    if prediction <= 0:
        prediction = 1   
    elif prediction >6:
        prediction = 6
    
    prediction = int(round(mean_impression + (wtd_sum/sum_wt)))
    print('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [70]:
predict_userbased(11676,'0060001453',impressions_matrix);


Predicted rating for user 11676 -> item 0060001453: 3


# Item-based Recommendation Systems

In [71]:
#This function finds k similar items given the item_id and impressions matrix
def findksimilaritems(item_id, impressions, metric=metric, k=k):
    similarities=[]
    indices=[]
    impressions=impressions.T
    loc = impressions.index.get_loc(item_id)
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute')
    model_knn.fit(impressions)
    
    distances, indices = model_knn.kneighbors(impressions.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()

    return similarities,indices

In [72]:
similarities,indices=findksimilaritems('0060001453',impressions_matrix)

In [73]:
#This function predicts the impression for specified user-item combination based on item-based approach
def predict_itembased(user_id, item_id, impressions, metric = metric, k=k):
    prediction= wtd_sum =0
    user_loc = impressions.index.get_loc(user_id)
    item_loc = impressions.columns.get_loc(item_id)
    similarities, indices=findksimilaritems(item_id, impressions) #similar users based on correlation coefficients
    sum_wt = np.sum(similarities)-1
    product=1
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] == item_loc:
            continue;
        else:
            product = impressions.iloc[user_loc,indices.flatten()[i]] * (similarities[i])
            wtd_sum = wtd_sum + product                              
    prediction = int(round(wtd_sum/sum_wt))
    
    #in case of very sparse datasets, using correlation metric for collaborative based approach may give negative ratings
    #which are handled here as below //code has been validated without the code snippet below, below snippet is to avoid negative
    #predictions which might arise in case of very sparse datasets when using correlation metric
    if prediction <= 0:
        prediction = 1   
    elif prediction >6:
        prediction = 6

    print('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))
    
    return prediction

In [74]:
prediction = predict_itembased(11676,'0060001453',impressions_matrix)


Predicted rating for user 11676 -> item 0060001453: 1


In [75]:
user_list=impressions_positive['user'].tolist()

In [76]:
user_list = sorted(list(set(user_list)))

In [77]:
prediction_matrix = impressions_matrix.copy()
prediction_matrix[prediction_matrix != 0] = 0

In [78]:
prediction_train = impressions_matrix.copy()
prediction_train[prediction_train != 0] = 0
for i in range(n_users):
    for j in range(n_books):
        userid = user_list[i]
        if (impressions_matrix[str(impressions_matrix.columns[j])][userid] != 0):
            prediction_train.values[i][j] = predict_itembased(userid, str(impressions_matrix.columns[j]),impressions_matrix)
            prediction_matrix.values[i][j] = predict_itembased(userid, str(impressions_matrix.columns[j]),impressions_matrix)
            
        else:
            prediction_matrix.values[i][j] = predict_itembased(userid, str(impressions_matrix.columns[j]),impressions_matrix)
MSE= mean_squared_error(prediction_train,impressions_matrix)
RMSE = round(sqrt(MSE),3)


Predicted rating for user 7346 -> item 0002005018: 1

Predicted rating for user 7346 -> item 000648302X: 1

Predicted rating for user 7346 -> item 0006550924: 1

Predicted rating for user 7346 -> item 0007110928: 1

Predicted rating for user 7346 -> item 0007154615: 1

Predicted rating for user 7346 -> item 0020198817: 1

Predicted rating for user 7346 -> item 002026478X: 1

Predicted rating for user 7346 -> item 0020518501: 1

Predicted rating for user 7346 -> item 002542730X: 1

Predicted rating for user 7346 -> item 0060001453: 1

Predicted rating for user 7346 -> item 0060002492: 1

Predicted rating for user 7346 -> item 006000438X: 1

Predicted rating for user 7346 -> item 0060004746: 1

Predicted rating for user 7346 -> item 0060005424: 1

Predicted rating for user 7346 -> item 0060009241: 1

Predicted rating for user 7346 -> item 0060083263: 1

Predicted rating for user 7346 -> item 0060085444: 1

Predicted rating for user 7346 -> item 0060092645: 1

Predicted rating for user 7


Predicted rating for user 7346 -> item 0312984820: 1

Predicted rating for user 7346 -> item 0312984839: 1

Predicted rating for user 7346 -> item 0312986769: 1

Predicted rating for user 7346 -> item 0312991460: 1

Predicted rating for user 7346 -> item 0312992416: 1

Predicted rating for user 7346 -> item 0316096199: 1

Predicted rating for user 7346 -> item 0316096199: 1

Predicted rating for user 7346 -> item 0316107352: 1

Predicted rating for user 7346 -> item 0316154059: 1

Predicted rating for user 7346 -> item 0316154075: 1

Predicted rating for user 7346 -> item 0316291161: 1

Predicted rating for user 7346 -> item 0316341118: 1

Predicted rating for user 7346 -> item 0316602906: 1

Predicted rating for user 7346 -> item 0316603570: 1

Predicted rating for user 7346 -> item 0316678104: 1

Predicted rating for user 7346 -> item 0316690619: 1

Predicted rating for user 7346 -> item 0316693006: 1

Predicted rating for user 7346 -> item 0316693235: 1

Predicted rating for user 7

Predicted rating for user 7346 -> item 0380757494: 1

Predicted rating for user 7346 -> item 0380759489: 2

Predicted rating for user 7346 -> item 0380759489: 2

Predicted rating for user 7346 -> item 0380761327: 1

Predicted rating for user 7346 -> item 0380762560: 1

Predicted rating for user 7346 -> item 0380762587: 1

Predicted rating for user 7346 -> item 0380764008: 1

Predicted rating for user 7346 -> item 0380767643: 1

Predicted rating for user 7346 -> item 0380771292: 1

Predicted rating for user 7346 -> item 0380772566: 1

Predicted rating for user 7346 -> item 0380774933: 1

Predicted rating for user 7346 -> item 0380776839: 1

Predicted rating for user 7346 -> item 0380776847: 1

Predicted rating for user 7346 -> item 0380777150: 1

Predicted rating for user 7346 -> item 0380780860: 1

Predicted rating for user 7346 -> item 0380782332: 1

Predicted rating for user 7346 -> item 0380784491: 1

Predicted rating for user 7346 -> item 0380786680: 1

Predicted rating for user 73


Predicted rating for user 7346 -> item 0440193613: 1

Predicted rating for user 7346 -> item 0440194784: 1

Predicted rating for user 7346 -> item 0440200563: 1

Predicted rating for user 7346 -> item 0440204208: 1

Predicted rating for user 7346 -> item 0440205573: 1

Predicted rating for user 7346 -> item 0440207770: 1

Predicted rating for user 7346 -> item 0440208149: 1

Predicted rating for user 7346 -> item 0440211263: 1

Predicted rating for user 7346 -> item 044021145X: 1

Predicted rating for user 7346 -> item 0440213320: 1

Predicted rating for user 7346 -> item 0440213991: 1

Predicted rating for user 7346 -> item 0440214114: 1

Predicted rating for user 7346 -> item 044021680X: 1

Predicted rating for user 7346 -> item 0440216842: 1

Predicted rating for user 7346 -> item 0440217563: 1

Predicted rating for user 7346 -> item 0440218535: 1

Predicted rating for user 7346 -> item 0440219337: 1

Predicted rating for user 7346 -> item 0440220793: 1

Predicted rating for user 7


Predicted rating for user 7346 -> item 0452269652: 1

Predicted rating for user 7346 -> item 0452272319: 1

Predicted rating for user 7346 -> item 0452272971: 1

Predicted rating for user 7346 -> item 0452281202: 1

Predicted rating for user 7346 -> item 0452281903: 1

Predicted rating for user 7346 -> item 0452282829: 1

Predicted rating for user 7346 -> item 0452284937: 1

Predicted rating for user 7346 -> item 0486264661: 1

Predicted rating for user 7346 -> item 0486264718: 1

Predicted rating for user 7346 -> item 0486266850: 1

Predicted rating for user 7346 -> item 0486270513: 1

Predicted rating for user 7346 -> item 0486275507: 1

Predicted rating for user 7346 -> item 0486275574: 1

Predicted rating for user 7346 -> item 0486278042: 1

Predicted rating for user 7346 -> item 0486280489: 1

Predicted rating for user 7346 -> item 0486424499: 1

Predicted rating for user 7346 -> item 050552208X: 1

Predicted rating for user 7346 -> item 0505523302: 1

Predicted rating for user 7

...


Predicted rating for user 265889 -> item 0399147888: 1

Predicted rating for user 265889 -> item 0399147926: 1

Predicted rating for user 265889 -> item 0399148027: 1

Predicted rating for user 265889 -> item 0399148450: 1

Predicted rating for user 265889 -> item 0399148515: 1

Predicted rating for user 265889 -> item 0399148582: 1

Predicted rating for user 265889 -> item 0399148612: 1

Predicted rating for user 265889 -> item 0399148639: 1

Predicted rating for user 265889 -> item 0399148701: 1

Predicted rating for user 265889 -> item 0399149368: 1

Predicted rating for user 265889 -> item 0399149759: 1

Predicted rating for user 265889 -> item 0399149783: 1

Predicted rating for user 265889 -> item 0399150811: 1

Predicted rating for user 265889 -> item 0399150870: 1

Predicted rating for user 265889 -> item 0399150900: 1

Predicted rating for user 265889 -> item 0399151850: 1

Predicted rating for user 265889 -> item 0399152288: 1

Predicted rating for user 265889 -> item 039923

Predicted rating for user 265889 -> item 0449005828: 1

Predicted rating for user 265889 -> item 0449005895: 1

Predicted rating for user 265889 -> item 0449005909: 1

Predicted rating for user 265889 -> item 0449006840: 6

Predicted rating for user 265889 -> item 0449006840: 6

Predicted rating for user 265889 -> item 0449148378: 1

Predicted rating for user 265889 -> item 0449148408: 1

Predicted rating for user 265889 -> item 0449205878: 1

Predicted rating for user 265889 -> item 0449206475: 1

Predicted rating for user 265889 -> item 0449208109: 1

Predicted rating for user 265889 -> item 0449208281: 1

Predicted rating for user 265889 -> item 0449209997: 1

Predicted rating for user 265889 -> item 0449210677: 1

Predicted rating for user 265889 -> item 0449213145: 1

Predicted rating for user 265889 -> item 0449213358: 1

Predicted rating for user 265889 -> item 0449213765: 1

Predicted rating for user 265889 -> item 0449213943: 1

Predicted rating for user 265889 -> item 0449216


Predicted rating for user 265889 -> item 0553573993: 6

Predicted rating for user 265889 -> item 0553574086: 1

Predicted rating for user 265889 -> item 0553574639: 1

Predicted rating for user 265889 -> item 0553575007: 1

Predicted rating for user 265889 -> item 0553575023: 1

Predicted rating for user 265889 -> item 0553575384: 1

Predicted rating for user 265889 -> item 0553575538: 1

Predicted rating for user 265889 -> item 0553576011: 1

Predicted rating for user 265889 -> item 0553577123: 1

Predicted rating for user 265889 -> item 055357759X: 1

Predicted rating for user 265889 -> item 0553578316: 1

Predicted rating for user 265889 -> item 0553579231: 5

Predicted rating for user 265889 -> item 0553579231: 5

Predicted rating for user 265889 -> item 0553579606: 1

Predicted rating for user 265889 -> item 0553580256: 1

Predicted rating for user 265889 -> item 0553580302: 1

Predicted rating for user 265889 -> item 055358068X: 1

Predicted rating for user 265889 -> item 055358


Predicted rating for user 265889 -> item 0743410181: 1

Predicted rating for user 265889 -> item 0743411269: 1

Predicted rating for user 265889 -> item 074341134X: 1

Predicted rating for user 265889 -> item 0743411366: 1

Predicted rating for user 265889 -> item 0743411374: 1

Predicted rating for user 265889 -> item 0743411943: 1

Predicted rating for user 265889 -> item 0743412621: 1

Predicted rating for user 265889 -> item 0743417593: 1

Predicted rating for user 265889 -> item 0743418190: 1

Predicted rating for user 265889 -> item 0743418204: 1

Predicted rating for user 265889 -> item 0743418220: 1

Predicted rating for user 265889 -> item 0743420373: 1

Predicted rating for user 265889 -> item 0743423585: 1

Predicted rating for user 265889 -> item 0743428188: 1

Predicted rating for user 265889 -> item 0743428587: 1

Predicted rating for user 265889 -> item 0743428617: 1

Predicted rating for user 265889 -> item 0743437128: 1

Predicted rating for user 265889 -> item 074343


Predicted rating for user 265889 -> item 1580082319: 1

Predicted rating for user 265889 -> item 1582341338: 1

Predicted rating for user 265889 -> item 1584855177: 5

Predicted rating for user 265889 -> item 1584855177: 5

Predicted rating for user 265889 -> item 1592400876: 1

Predicted rating for user 265889 -> item 1841721522: 1

Predicted rating for user 265889 -> item 1853260150: 1

Predicted rating for user 265889 -> item 1853260207: 1

Predicted rating for user 265889 -> item 1853260436: 1

Predicted rating for user 265889 -> item 1853260789: 1

Predicted rating for user 265889 -> item 1853261580: 1

Predicted rating for user 265889 -> item 1853262447: 1

Predicted rating for user 265889 -> item 1857027051: 1

Predicted rating for user 265889 -> item 1860461670: 1

Predicted rating for user 265889 -> item 1885171080: 1

Predicted rating for user 265889 -> item 1889392006: 1

Predicted rating for user 265889 -> item 1892723018: 1

Predicted rating for user 265889 -> item 193156

In [79]:
print("RMSE is: {0}".format(RMSE))

RMSE is: 0.417


In [80]:
userid=104636
count=0
for j in range(n_books):
    if (count<3):
        if (prediction_matrix[str(prediction_matrix.columns[j])][userid] == 6):
            if (impressions_matrix[str(impressions_matrix.columns[j])][userid] != 6): 
                book=str(prediction_matrix.columns[j])
                count=count+1
                print ('\nUser: {0} is most likely to purchase book {1} next'.format(userid,book))


User: 104636 is most likely to purchase book 0380820447 next

User: 104636 is most likely to purchase book 0440193613 next

User: 104636 is most likely to purchase book 044022165X next


In [81]:
prediction_matrix.to_csv('D:/userdata/girims/Downloads/ShopBonsai/output.csv')